In [1]:
"""
Three cases used by the model: winter (case1), fall turnover (case 2), stratified season (case 3).
For each case, computation of the temperatures, solids concentration and thermocline
depth at the next time step. 

TO FILL!
"""

import numpy as np
import sys
from lake_functions import alphaT,rho_TCss, seasonal_therm
from statistics import mean

def case1(VAR_LAKE,Twinter=4,hmax=8):
    """
    Winter case: ice-covered lake, no heat fluxes = constant temperatures 
    and concentrations. 
     
    INPUTS:
        VAR_LAKE: dictionary containing
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C]
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L]
            h_epi: thermocline depth [m]
        Twinter: temperature of maximum density [°C]
        hmax: lake depth [m]
        
    OUTPUTS:
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step
    """

    # -------- CODE --------
    # Temperature
    T_epi1 = Twinter
    T_hypo1 = Twinter
    # solids concnetrations
    Css_epi1 = 100
    Css_hypo1 = 100

    h_epi1 = hmax
    
    VAR_LAKE={"T_epi":T_epi1,"T_hypo":T_hypo1,"Css_epi":Css_epi1,"Css_hypo":Css_hypo1,"h_epi":h_epi1}

    return VAR_LAKE

In [2]:

def case2(VAR_LAKE,Hsurf,Hsw0,tyear,iceon,Vs,C_FFT,hmax=8,g=9.81,Cpw=4200,Twinter=4,A0=7.8*10**6):
    """
    Fall turnover case: one box with heat fluxes.
     
    INPUTS:
        VAR_LAKE: dictionary containing
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C]
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L]
            h_epi: thermocline depth [m]
        Hsurf: surface heat flux [W/m^2]
        Hsw0: surface radiation flux [W/m^2]
        tyear: time as DOY
        iceon: iceon date as DOY
        Vs: settling velocity [m/s]
        C_FFT: solids concentration in the FFT [g/m^3]
        hmax: lake depth [m]
        g: gravitational acceleration [m/s^2]
        Cpw: heat capacity of water [J.kg^(-1).K^(-1)]
        Twinter: temperature of maximum density [°C]
        A0: lake surface area [m^2]
        
        
    OUTPUTS:
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step
        iceon: iceon date as DOY (modified only if ice formation after initial iceon date)
    """
    # -------- CODE --------
    # ----Initial setup----
    h_epi2 = hmax  # Thermocline depth
    T2 = VAR_LAKE["T_epi"]  # Temperature, constant
    Css2 = VAR_LAKE["Cssepi"] # Initial solid concentration constant
    rho_w2 = rho_TCss(T_epi2, Css_epi2) # total density
    m_w2 = rho_w2*A0*hmax # water mass
    
    # ----Special process in fall----
    # Heat flux and temperature change
    Hsurf = surfheat(T2,Ta,Wsp,RH,P,C)
    Qnet2 = -(Hsurf + Hsw0)*A0
    dT2 = (Qnet2/(m_w2*Cpw))/24*60*60
    Tnew2 = T2 + dT2

    # --turbulent mixing--
    C_sed = -Css*Vs*A0*24*60*60/A0*hmax
    # --Redistribution--
    alpha2 = alphaT(T2)
    B = alpha2*g*(H_surf + Hsw0*(1-np.exp((0.325+66*Css2)*hmax))) / (rho_w2 * Cpw)
    # velocity
    rho_30 = rho_TCss(T_epi2, 30)
    N_fft2 = g*(rho_30-rho_w2)/(0.5*rho_FFT) # WHAT IS RHOFFT!!!!
    Vr = (1+2*0.2)*B/(N_fft2*hmax)
    
    C_red = C_FFT*Vr*A0*24*60*60/A0*hmax

    Cssnew2 = Css2 + C_sed + C_red
    
    # iceon (not sure)
    if T_epi < Twinter:
        iceon = tyear

    VAR_LAKE={"T_epi":Tnew2,"T_hypo":Tnew2,"Css_epi":Css2,"Css_hypo":Css2,"h_epi":h_epi2}
    return VAR_LAKE, iceon

In [ ]:
def case3(VAR_LAKE,Hsurf,Hsw0,tyear,iceon,Vs,ustar,hmax=8,g=9.81,Cpw=4200,Twinter=4,A0=7.8*10**6,Dth=1.4E-7):
    """
    spring-summer, 2 boxes.
     
    INPUTS:
        VAR_LAKE: dictionary containing
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C]
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L]
            h_epi: thermocline depth [m]
        Hsurf: surface heat flux [W/m^2]
        Hsw0: surface radiation flux [W/m^2]
        tyear: time as DOY
        iceon: iceon date as DOY
        Vs: settling velocity [m/s]
        ustar: friction velocity [m/s]
        hmax: lake depth [m]
        g: gravitational acceleration [m/s^2]
        Cpw: heat capacity of water [J.kg^(-1).K^(-1)]
        Twinter: temperature of maximum density [°C]
        A0: lake surface area [m^2]
        Dth: thermal molecular diffusivity [m^2/s]
        
  
    OUTPUTS:
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step
        iceon: iceon date as DOY (modified only if ice formation after initial iceon date)
    """
    # --- thermocline depth --- 
    h_epi3 = seasonal_therm(prev_h_epi, lake_params["dt"], fluxes["B"], N2)

    # --- epilimnion temperature ---
    # ΔT = (Q_surf + Q_sw) * dt / (rho * Cp * volume of epilimnion)
    # surface heat fluxes from def surfheat() include only sensible heat, latent heat, longwave radiation
    # Beer_lambert law computes short wave radiation
    T_epi3 = VAR_LAKE["T_epi"] + ((fluxes["H_surf"] + fluxes["H_sw_abs"]) * lake_params["dt"] / (lake_params["rho"] * lake_params["Cp"] * h_epi3))

     # --- hypolimnion temperature ---- 
    # vertical diffusive flux 𝐻_diff = −𝜌𝐶𝑝𝑤(𝐷𝑡h +𝐾𝑧)dTdz
    dTdz = (VAR_LAKE["T_epi"] - VAR_LAKE["T_hypo"]) / (0.5 * lake_params["hmax"]) # temperature gradient
    H_diff = -lake_params["rho"] * lake_params["Cp"] * lake_params["Kz"] * dTdz # vertical diffusive flux
    T_hypo3 = VAR_LAKE["T_hypo"] + (H_diff * lake_params["dt"]/ (lake_params["rho"] * lake_params["Cp"] * (lake_params["hmax"] - h_epi3))) # ΔT = (H_diff⋅dt)/(ρ⋅Cp⋅h_epi)

    # --- particle settling from epilimnion to hypolimnion ----
    # F_sed = Css * v_sed * A0 (affects both epilimnion and hypolimnion) 
    F_sed = VAR_LAKE["Css_epi"] * lake_params["vs"] * lake_params["A0"] # solids leaving the epilimnion [mg/s] 
    # ΔC = ΔM/V = (F_sed⋅Δt) / (A0⋅h_epi) → gives correct units [mg/L]
    Css_epi3 = VAR_LAKE["Css_epi"] - F_sed * lake_params["dt"] / (lake_params["A0"] * h_epi3) # updates the epilimnion concentration
    Css_hypo3 = VAR_LAKE["Css_hypo"] + F_sed * lake_params["dt"] / (lake_params["A0"] * (lake_params["hmax"] - h_epi3)) # updates the hypolimnion concentration
    
    # --- particle redistribution due to thermocline change ----
    # F_red = Css * A0 * dh/dt
    # volume of both boxes changes → water moves between epi and hypo → mass of suspended solid changes
    dh = h_epi3 - prev_h_epi # change in thermocline depth per time step
    if dh < 0:  # thermocline shallows → epi water moves to hypo
        F_red = VAR_LAKE["Css_epi"] * lake_params["A0"] * abs(dh)
        Css_epi3 -= F_red / (lake_params["A0"] * h_epi3)) # mass removed from the new, smaller hypo
        Css_hypo3 += F_red / (lake_params["A0"] * (lake_params["hmax"] - h_epi3) # mass added to new, larger epi

    elif dh > 0:  # thermocline deepens → hypo water moves to epi
        F_red = VAR_LAKE["Css_hypo"] * lake_params["A0"] * dh
        Css_hypo3 -= F_red / (lake_params["A0"] * (lake_params["hmax"] - h_epi3)
        Css_epi3 += F_red / (lake_params["A0"] * h_epi3))
 # --- returns updated state ----
    return {"T_epi": T_epi3, "T_hypo": T_hypo3, "Css_epi": Css_epi3,"Css_hypo": Css_hypo3,"h_epi": h_epi3}